# Import Required Libraries
Import all necessary libraries such as numpy, pandas, keras, and sklearn.

In [2]:
# Import Required Libraries
import numpy as np
import pandas as pd
import os
import re
import shutil
import random
from glob import glob
from collections import defaultdict

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold

import cv2
from PIL import Image
matplotlib.use('Agg')   # type: ignore
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from sklearn.metrics import confusion_matrix
from keras.models import load_model

# Define CNN Model
Define the `cnn_model` function to create and compile the CNN model.

In [3]:
# Define CNN Model
def cnn_model(input_shape, num_classes):
	model = Sequential([
		Conv2D(32, (5,5), activation='relu', input_shape=input_shape),
		MaxPooling2D(2,2),
		Conv2D(64, (5,5), activation='relu'),
		MaxPooling2D(2,2),
		Flatten(),
		Dense(128, activation='relu'),
		Dense(num_classes, activation='softmax')
	])
	model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
	return model

# Dataset Preparation and Undersampling
Prepare the dataset by undersampling classes to balance the data and create a DataFrame for training and validation.

In [4]:
# Define dataset directory and parameters
images_dir = "combined-dataset"
img_size = (32, 32)
batch_size = 32
epochs = 15
k_folds = 5
cross_val_accuracies = {}

In [25]:
# Dataset Preparation and Undersampling
def prepare_dataset(method_path):
	class_dirs = [d for d in os.listdir(method_path) if os.path.isdir(os.path.join(method_path, d))]
	class_to_images = defaultdict(list)
	for class_name in class_dirs:
		class_path = os.path.join(method_path, class_name)
		image_paths = glob(os.path.join(class_path, "*"))
		class_to_images[class_name].extend(image_paths)

	target_size = min(
		max(1000 if k in ['Euro-American', 'East Asian'] else len(v), len(v))
		for k, v in class_to_images.items()
	)
	balanced_image_paths = []
	balanced_labels = []
	for cls, imgs in class_to_images.items():
		sampled = random.sample(imgs, target_size) if len(imgs) > target_size else imgs
		balanced_image_paths.extend(sampled)
		balanced_labels.extend([cls] * len(sampled))

	combined = list(zip(balanced_image_paths, balanced_labels))
	random.shuffle(combined)
	balanced_image_paths, balanced_labels = zip(*combined)
	df = pd.DataFrame({'filename': balanced_image_paths, 'class': balanced_labels})
	return df

# K-Fold Cross-Validation for Resolution Folders
Perform K-Fold cross-validation on the resolution folders, train the CNN model, and save the trained models.

In [ ]:
# K-Fold Cross-Validation for Resolution Folders
for resolution_folder in sorted(os.listdir(images_dir), key=lambda x: int(re.search(r'\d+', x).group())):
	resolution_path = os.path.join(images_dir, resolution_folder)
	if os.path.isdir(resolution_path):
		for method_folder in os.listdir(resolution_path):
			method_path = os.path.join(resolution_path, method_folder)
			if os.path.isdir(method_path):
				print(f"\nCross-validating {resolution_folder}/{method_folder}...")
				df = prepare_dataset(method_path)

				datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
				train_data = datagen.flow_from_dataframe(
					dataframe=df, x_col='filename', y_col='class',
					target_size=img_size, batch_size=batch_size,
					class_mode='categorical', subset='training', shuffle=True
				)
				val_data = datagen.flow_from_dataframe(
					dataframe=df, x_col='filename', y_col='class',
					target_size=img_size, batch_size=batch_size,
					class_mode='categorical', subset='validation', shuffle=False
				)

				num_samples = len(df)
				num_classes = len(df['class'].unique())
				input_shape = (img_size[0], img_size[1], 3)

				kfold = KFold(n_splits=k_folds, shuffle=True)
				fold_accuracies = []

				for train_idx, val_idx in kfold.split(np.arange(num_samples)):
					model = cnn_model(input_shape, num_classes)
					history = model.fit(train_data, validation_data=val_data, epochs=epochs, verbose=1)
					max_fold_acc = max(history.history['val_accuracy'])
					fold_accuracies.append(max_fold_acc)
					print(f"Fold accuracy: {max_fold_acc:.4f}")

				model_path = os.path.join("models/kfold", resolution_folder, method_folder)
				os.makedirs(model_path, exist_ok=True)
				model_name = f"{resolution_folder}_{method_folder}_fold_{len(fold_accuracies)}_mask_5_aug-under.h5"
				model.save(os.path.join(model_path, model_name))
				print(f"Model saved to {model_path}")

				avg_acc = np.mean(fold_accuracies)
				key = f"{resolution_folder} - {method_folder}"
				cross_val_accuracies[key] = avg_acc
				print(f"Average accuracy for {key}: {avg_acc:.4f}")

# Model Evaluation
Evaluate the trained models on the test set and save the evaluation results.

In [27]:
# Extract and sort resolutions numerically
resolutions = sorted(set(k.split(" - ")[0] for k in cross_val_accuracies.keys()), key=lambda x: int(x.replace("px", "")))
methods = sorted(set(k.split(" - ")[1] for k in cross_val_accuracies.keys()))

# Organize data for plotting
data = {method: [cross_val_accuracies.get(f"{res} - {method}", None) for res in resolutions] for method in methods}

# Define a color map for better distinction
colors = plt.cm.get_cmap("tab10", len(methods))  # Use "tab10" color map with enough colors

# Plot the lines
plt.figure(figsize=(16, 10))
for i, (method, accuracies) in enumerate(data.items()):
	plt.plot(resolutions, accuracies, marker="o", linestyle="-", linewidth=2, markersize=8, label=method, color=colors(i))

# Labels and title
plt.xlabel("Resolution", fontsize=12)
plt.ylabel("Max Accuracy", fontsize=12)
plt.title("CNN Validation Accuracy by Resolution and Method", fontsize=14)

# Move legend outside the plot for better clarity
plt.legend(title="Method", bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=10)

# Add a grid with transparency
plt.grid(True, linestyle="--", alpha=0.6)

# Improve layout to fit legend properly
plt.tight_layout()

# Show plot
plt.show()

plt.savefig("../../imgs/graphs/kfold-undersample/cnn_validation_accuracy_mask_5-kfold_aug-under.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_18440\3227416947.py:9: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = plt.cm.get_cmap("tab10", len(methods))  # Use "tab10" color map with enough colors
C:\Users\theof\AppData\Local\Temp\ipykernel_18440\3227416947.py:31: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [28]:
# Extract and sort resolutions numerically
resolutions = sorted(set(k.split(" - ")[0] for k in cross_val_accuracies.keys()), key=lambda x: int(x.replace("px", "")))

# Extract unique methods and split into two groups of 5
methods = sorted(set(k.split(" - ")[1] for k in cross_val_accuracies.keys()))
methods_group1 = methods[:5]
methods_group2 = methods[5:]

# Organize data for plotting
data1 = {method: [cross_val_accuracies.get(f"{res} - {method}", None) for res in resolutions] for method in methods_group1}
data2 = {method: [cross_val_accuracies.get(f"{res} - {method}", None) for res in resolutions] for method in methods_group2}

# Create subplots
fig, axs = plt.subplots(2, 1, figsize=(10, 10), sharex=True, sharey=True)

colors = plt.cm.get_cmap("tab10", len(methods_group1))  # Use "tab10" color map with enough colors

# Plot first group
for method, accuracies in data1.items():
	axs[0].plot(resolutions, accuracies, marker="o", label=method, color=colors(methods_group1.index(method)))
axs[0].set_title("CNN Validation Accuracy (Group 1)")
axs[0].set_ylabel("Max Accuracy")
axs[0].legend(title="Method")
axs[0].grid(True, linestyle="--", alpha=0.6)

# Plot second group
for method, accuracies in data2.items():
	axs[1].plot(resolutions, accuracies, marker="o", label=method, color=colors(methods_group2.index(method)))
axs[1].set_title("CNN Validation Accuracy (Group 2)")
axs[1].set_xlabel("Resolution")
axs[1].set_ylabel("Max Accuracy")
axs[1].legend(title="Method")
axs[1].grid(True, linestyle="--", alpha=0.6)

# Adjust layout
plt.tight_layout()
plt.show()

plt.savefig("../../imgs/graphs/kfold-undersample/cnn_validation_accuracy_groups_mask_5_kfold_aug-under.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_18440\1357055831.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = plt.cm.get_cmap("tab10", len(methods_group1))  # Use "tab10" color map with enough colors
C:\Users\theof\AppData\Local\Temp\ipykernel_18440\1357055831.py:37: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [7]:
# confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from keras.models import load_model

# Load the model
model_path = 'models/kfold/100px/Chargaff-Diversite-NucleScore/100px_Chargaff-Diversite-NucleScore_fold_5_mask_5_undersample.h5'
model = load_model(model_path)
model.summary()

# Load the test data
test_data_dir = 'test-raw/arrays/100px/Chargaff-Diversite-NucleScore'

# Ensure correct image size for this model
img_size = (32, 32)

test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(
	test_data_dir,
	target_size=img_size,
	batch_size=batch_size,
	class_mode='categorical',
	shuffle=False
)
# Get the true labels
true_labels = test_data.classes
class_labels = list(test_data.class_indices.keys())

# Get the predicted labels
predictions = model.predict(test_data, steps=len(test_data), verbose=1)
predicted_labels = np.argmax(predictions, axis=1)
# Generate the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Confusion Matrix for CNN Model (100px Chargaff-Diversite-NucleScore)')
plt.show()

plt.savefig("../../imgs/graphs/kfold-undersample/cnn_confusion_matrix_100px_mask_5-kfold_undersample.png")

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_214"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_428 (Conv2D)             │ (None, 28, 28, 32)     │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_428               │ (None, 14, 14, 32)     │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_429 (Conv2D)             │ (None, 10, 10, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_429               │ (None, 5, 5, 64)       │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_214 (Flatten)           │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_428 (Dense)               │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_429 (Dense)               │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 259,271 (1012.78 KB)

 Trainable params: 259,269 (1012.77 KB)

...

 Optimizer params: 2 (12.00 B)

Found 679 images belonging to 5 classes.
 1/22 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step


C:\Users\theof\AppData\Local\Temp\ipykernel_11396\1187647354.py:41: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [30]:
train_files = set(train_data.filepaths)
val_files = set(val_data.filepaths)
test_files = set(test_data.filepaths)  # Assuming you've created this

# Check for overlap
print("Train ∩ Val:", len(train_files & val_files), "(" + str(len(train_files & val_files) / len(train_files) * 100) + "%)")
print("Train ∩ Test:", len(train_files & test_files), "("+ str(len(train_files & test_files) / len(train_files) * 100) + "%)")
print("Val ∩ Test:", len(val_files & test_files), "("+ str(len(val_files & test_files) / len(val_files) * 100) + "%)")

Train ∩ Val: 0 (0.0%)
Train ∩ Test: 1668 (100.0%)
Val ∩ Test: 417 (100.0%)


# K-Fold Cross-Validation for Mosaic Folders
Perform K-Fold cross-validation on the mosaic folders, train the CNN model, and save the trained models.

In [5]:
# Define dataset directory
mosaics_dir = 'combined-dataset-mosaics'

# Training parameters
img_size = (20, 50)  # Resize images to 64x64
batch_size = 32
epochs = 30

k_folds = 10  # Number of folds for cross-validation

# Dictionary to store max accuracies
cross_val_accuracies_mos = {}

In [32]:
for resolution_folder in sorted(os.listdir(mosaics_dir), key=lambda x: int(re.search(r'\d+', x).group())):
	resolution_path = os.path.join(mosaics_dir, resolution_folder)
	if os.path.isdir(resolution_path):
		print(f"\nCross-validating {resolution_folder}...")
		df = prepare_dataset(resolution_path)

		datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
		train_data = datagen.flow_from_dataframe(
			dataframe=df, x_col='filename', y_col='class',
			target_size=img_size, batch_size=batch_size,
			class_mode='categorical', subset='training', shuffle=True
		)
		val_data = datagen.flow_from_dataframe(
			dataframe=df, x_col='filename', y_col='class',
			target_size=img_size, batch_size=batch_size,
			class_mode='categorical', subset='validation', shuffle=True
		)

		num_samples = len(df)
		num_classes = len(df['class'].unique())
		input_shape = (img_size[0], img_size[1], 3)

		kfold = KFold(n_splits=k_folds, shuffle=True)
		fold_accuracies = []

		for train_idx, val_idx in kfold.split(np.arange(num_samples)):
			model = cnn_model(input_shape, num_classes)
			history = model.fit(train_data, validation_data=val_data, epochs=epochs, verbose=1)
			max_fold_acc = max(history.history['val_accuracy'])
			fold_accuracies.append(max_fold_acc)
			print(f"Fold accuracy: {max_fold_acc:.4f}")

		model_path = os.path.join("models/kfold_mosaic", resolution_folder)
		os.makedirs(model_path, exist_ok=True)
		model_name = f"{resolution_folder}_fold_{len(fold_accuracies)}_mosaic_mask_5_aug-under.h5"
		model.save(os.path.join(model_path, model_name))
		print(f"Model saved to {model_path}")

		avg_acc = np.mean(fold_accuracies)
		cross_val_accuracies_mos[resolution_folder] = avg_acc
		print(f"Average accuracy for {resolution_folder}: {avg_acc:.4f}")


Cross-validating 4px...
Found 888 validated image filenames belonging to 5 classes.
Found 222 validated image filenames belonging to 5 classes.
Epoch 1/30
28/28 [==============================] - 1s 25ms/step - loss: 1.6221 - accuracy: 0.2354 - val_loss: 1.6127 - val_accuracy: 0.2027
Epoch 2/30
28/28 [==============================] - 1s 19ms/step - loss: 1.6094 - accuracy: 0.2218 - val_loss: 1.6048 - val_accuracy: 0.2297
Epoch 3/30
28/28 [==============================] - 1s 18ms/step - loss: 1.5960 - accuracy: 0.2872 - val_loss: 1.6004 - val_accuracy: 0.2072
Epoch 4/30
28/28 [==============================] - 1s 18ms/step - loss: 1.5770 - accuracy: 0.2793 - val_loss: 1.5590 - val_accuracy: 0.2703
Epoch 5/30
28/28 [==============================] - 0s 17ms/step - loss: 1.5426 - accuracy: 0.3086 - val_loss: 1.5359 - val_accuracy: 0.3333
Epoch 6/30
28/28 [==============================] - 0s 17ms/step - loss: 1.5099 - accuracy: 0.3435 - val_loss: 1.5140 - val_accuracy: 0.3153
Epoch 7/3

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 224 validated image filenames belonging to 5 classes.
Epoch 1/30
28/28 [==============================] - 1s 31ms/step - loss: 1.6066 - accuracy: 0.2511 - val_loss: 1.5903 - val_accuracy: 0.2634
Epoch 2/30
28/28 [==============================] - 1s 18ms/step - loss: 1.5475 - accuracy: 0.3214 - val_loss: 1.5473 - val_accuracy: 0.2366
Epoch 3/30
28/28 [==============================] - 1s 18ms/step - loss: 1.4484 - accuracy: 0.3761 - val_loss: 1.4804 - val_accuracy: 0.3080
Epoch 4/30
28/28 [==============================] - 0s 17ms/step - loss: 1.3312 - accuracy: 0.4665 - val_loss: 1.3632 - val_accuracy: 0.4821
Epoch 5/30
28/28 [==============================] - 0s 17ms/step - loss: 1.2379 - accuracy: 0.5134 - val_loss: 1.2570 - val_accuracy: 0.5179
Epoch 6/30
28/28 [==============================] - 0s 17ms/step - loss: 1.0881 - accuracy: 0.5960 - val_loss: 1.0921 - val_accuracy: 0.5759
Epoch 7/30
28/28 [==============================] - 0s 17ms/step - loss: 1.0307 - accuracy: 0.

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 224 validated image filenames belonging to 5 classes.
Epoch 1/30
28/28 [==============================] - 1s 32ms/step - loss: 1.6195 - accuracy: 0.2310 - val_loss: 1.6004 - val_accuracy: 0.2768
Epoch 2/30
28/28 [==============================] - 1s 19ms/step - loss: 1.5568 - accuracy: 0.2924 - val_loss: 1.5290 - val_accuracy: 0.2054
Epoch 3/30
28/28 [==============================] - 1s 18ms/step - loss: 1.4234 - accuracy: 0.4029 - val_loss: 1.3572 - val_accuracy: 0.4420
Epoch 4/30
28/28 [==============================] - 0s 17ms/step - loss: 1.2756 - accuracy: 0.4821 - val_loss: 1.2351 - val_accuracy: 0.5179
Epoch 5/30
28/28 [==============================] - 0s 17ms/step - loss: 1.1212 - accuracy: 0.5257 - val_loss: 1.1003 - val_accuracy: 0.5982
Epoch 6/30
28/28 [==============================] - 0s 17ms/step - loss: 1.0574 - accuracy: 0.5446 - val_loss: 1.0269 - val_accuracy: 0.6250
Epoch 7/30
28/28 [==============================] - 0s 17ms/step - loss: 0.9014 - accuracy: 0.

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 224 validated image filenames belonging to 5 classes.
Epoch 1/30
28/28 [==============================] - 1s 33ms/step - loss: 1.5942 - accuracy: 0.2790 - val_loss: 1.5410 - val_accuracy: 0.2634
Epoch 2/30
28/28 [==============================] - 1s 21ms/step - loss: 1.5168 - accuracy: 0.3382 - val_loss: 1.4974 - val_accuracy: 0.3973
Epoch 3/30
28/28 [==============================] - 1s 19ms/step - loss: 1.4119 - accuracy: 0.4330 - val_loss: 1.2957 - val_accuracy: 0.5045
Epoch 4/30
28/28 [==============================] - 1s 19ms/step - loss: 1.1894 - accuracy: 0.5446 - val_loss: 1.0863 - val_accuracy: 0.6295
Epoch 5/30
28/28 [==============================] - 1s 20ms/step - loss: 1.0152 - accuracy: 0.6060 - val_loss: 1.0339 - val_accuracy: 0.6027
Epoch 6/30
28/28 [==============================] - 1s 18ms/step - loss: 0.8595 - accuracy: 0.6942 - val_loss: 0.8207 - val_accuracy: 0.7321
Epoch 7/30
28/28 [==============================] - 1s 19ms/step - loss: 0.7277 - accuracy: 0.

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 888 validated image filenames belonging to 5 classes.
Found 222 validated image filenames belonging to 5 classes.
Epoch 1/30
28/28 [==============================] - 1s 33ms/step - loss: 1.5592 - accuracy: 0.3041 - val_loss: 1.5553 - val_accuracy: 0.2703
Epoch 2/30
28/28 [==============================] - 1s 20ms/step - loss: 1.4399 - accuracy: 0.3851 - val_loss: 1.4449 - val_accuracy: 0.3288
Epoch 3/30
28/28 [==============================] - 1s 19ms/step - loss: 1.3849 - accuracy: 0.4110 - val_loss: 1.3340 - val_accuracy: 0.4550
Epoch 4/30
28/28 [==============================] - 1s 19ms/step - loss: 1.2274 - accuracy: 0.5101 - val_loss: 1.2105 - val_accuracy: 0.4775
Epoch 5/30
28/28 [==============================] - 1s 19ms/step - loss: 1.0404 - accuracy: 0.6104 - val_loss: 0.9568 - val_accuracy: 0.6802
Epoch 6/30
28/28 [==============================] - 1s 18ms/step - loss: 0.8683 - accuracy: 0.7016 - val_loss: 0.8158 - val_accuracy: 0.6757
Epoch 7/30
28/28 [================

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [33]:
# Extract and sort resolutions numerically
resolutions = sorted(cross_val_accuracies_mos.keys(), key=lambda x: int(x.replace("px", "")))

# Get max accuracy values in the correct order
accuracies = [cross_val_accuracies_mos[res] for res in resolutions]

# Plot the accuracy per resolution
plt.figure(figsize=(12, 6))
plt.plot(resolutions, accuracies, marker="o", linestyle="-", linewidth=2, markersize=8, color="blue", label="Accuracy")

# Labels and title
plt.xlabel("Resolution", fontsize=12)
plt.ylabel("Max Accuracy", fontsize=12)
plt.title("CNN Validation Accuracy by Resolution", fontsize=14)

# Add data points on the plot
for i, acc in enumerate(accuracies):
	plt.text(resolutions[i], acc, f"{acc:.2f}", fontsize=10, ha="right")

# Grid and legend
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend(loc="lower right")

# Save and show plot
plt.tight_layout()
plt.show()
plt.savefig("../../imgs/graphs/kfold-undersample/cnn_validation_accuracy_kfold_mosaics_line_mask_5_aug-under.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_18440\1139481552.py:26: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [8]:
# confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from keras.models import load_model

# Load the model for the new version
new_model_path = "models/kfold_mosaic/100px/100px_fold_10_mosaic_mask_5_undersample.h5"
new_model = load_model(new_model_path)
new_model.summary()

# Load the test data for the new version
new_test_data_dir = 'test-raw/mosaics/100px'

new_test_datagen = ImageDataGenerator(rescale=1./255)
new_test_data = new_test_datagen.flow_from_directory(
	new_test_data_dir,
	target_size=img_size,
	batch_size=batch_size,
	class_mode='categorical',
	shuffle=False
	)

# Get the true labels for the new version
new_true_labels = new_test_data.classes
new_class_labels = list(new_test_data.class_indices.keys())

# Get the predicted labels for the new version
new_predictions = new_model.predict(new_test_data, steps=len(new_test_data), verbose=1)
new_predicted_labels = np.argmax(new_predictions, axis=1)

# Generate the confusion matrix for the new version
new_cm = confusion_matrix(new_true_labels, new_predicted_labels)

# Plot the confusion matrix for the new version
plt.figure(figsize=(10, 8))
sns.heatmap(new_cm, annot=True, fmt='d', cmap='Blues', xticklabels=new_class_labels, yticklabels=new_class_labels)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Confusion Matrix for CNN Model (100px Mosaics)')
plt.show()

plt.savefig("../../imgs/graphs/kfold-undersample/cnn_confusion_matrix_kfold_mosaics_100px_mask_5_undersample.png")

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_322"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_644 (Conv2D)             │ (None, 16, 46, 32)     │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_644               │ (None, 8, 23, 32)      │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_645 (Conv2D)             │ (None, 4, 19, 64)      │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_645               │ (None, 2, 9, 64)       │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_322 (Flatten)           │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_644 (Dense)               │ (None, 128)            │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_645 (Dense)               │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 201,927 (788.78 KB)

 Trainable params: 201,925 (788.77 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

Found 679 images belonging to 5 classes.


c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_644" is incompatible with the layer: expected axis -1 of input shape to have value 1152, but received input with shape (32, 1600)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 32, 32, 3), dtype=float32)
  • training=False
  • mask=None
  • kwargs=<class 'inspect._empty'>

In [35]:
new_train_files = set(train_data.filepaths)
new_val_files = set(val_data.filepaths)
new_test_files = set(new_test_data.filepaths)

# Check for overlap
print("Train ∩ Val:", len(new_train_files & new_val_files), "(" + str(len(new_train_files & new_val_files) / len(new_train_files) * 100) + "%)")
print("Train ∩ Test:", len(new_train_files & new_test_files), "("+ str(len(new_train_files & new_test_files) / len(new_train_files) * 100) + "%)")
print("Val ∩ Test:", len(new_val_files & new_test_files), "("+ str(len(new_val_files & new_test_files) / len(new_val_files) * 100) + "%)")


Train ∩ Val: 0 (0.0%)
Train ∩ Test: 888 (100.0%)
Val ∩ Test: 222 (100.0%)
